<a href="https://colab.research.google.com/github/Arsiry/pet-project-adtech/blob/main/pet_project_adtech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pet Project from AdTech Domain

## Description of Project

**Real time Advertiser's Auction**

60% of the digital ad inventory is sold by publishers in Real Time first price Auctions.

Once a user lands on a webpage, bidders (advertisers) bid for different ad slots on the page and the one with the highest winning bid displays their ad in the ad space and pays the amount he bid. This process encourages bid shading – bidding lesser than the perceived value of the ad space to maximize utilization for self while maintaining a particular win rate at lowest prices.

Hence, for publishers, it becomes important to value their inventory (all the users that visit their website * all the ad slots they have on their websites) correctly so that a reserve price, or a minimum price can be set up for the auctions.

In a first price auction, the highest bidder wins and pays the price they bid if it exceeds the reserve price. The optimal strategy of a bidder is to shade their bids (bid less than their true value of the inventory). However, bidder needs to win a certain amount to achieve their goals. This suggests they need to shade as much possible while maintaining a certain win rate.

A bidder perceives a certain value out of every impression they win. Each bidder would like to maintain the value they derived out of this set of websites (given in the dataset) in June with a maximum deviation of 20%.

Setting a reserve price induces this by causing bidders to lose at lower bids which encourages higher bidding and more publisher revenue. However, since most of these takes place through automated systems, there might be an unknown delay in setting reserve prices & reducing win rate of bidder & bidder changing their bid shading algorithm & increased publisher revenue.

IMPORTANT TERMS:
- o Publisher – person who owns and publishes content on the website
- o Inventory – all the users that visit the website * all the ad slots present in the website for the observation period
- o Impressions - showing an ad to a user constitutes one impression. If the ad slot is present but an ad is not shown, it falls as “unfilled impression”. Inventory is the sum of impressions + unfilled impressions.
- o CPM – cost per Mille. This is one of the most important ways to measure performance. It is. Calculated as revenue/impressions * 1000. 'bids' and 'price' are measured in terms of CPM

## Description of DataSet

The dataset provided to you has data for several websites owned by the same company and they are asking for your help for what should be their approach to set reserve prices and what is the range for reserve prices they should be setting for July. The data is only of the actual revenue generation and not at bid level. The dataset has the following columns:

- site_id : each id denotes a different website
- ad_type_id : each id denotes a different ad_type. These can be display ads , video ads, text ads etc
- geo_id : each id denotes a different country. our maximum traffic is from english speaking countries
- device_category_id : each id denoted a different device_category like desktop , mobile, tablet
- advertiser_id: each id denotes a different bidder in the auction
- order_id : can be ignored
- line_item_type_id : can be ignored
- os_id : each id denotes a different operating system for mobile device category only (android , ios etc) . - for all other device categories, os_id will correspond to not_mobile
- integration_type_id : it describes how the demand partner is setup within a publisher's ecosystem - can be - adserver (running through the publisher adserver) or hardcoded
- monetization_channel_id : it describes the mode through which demand partner integrates with a particular - - publisher - it can be header bidding (running via prebid.js), dynamic allocation, exchange bidding, direct etc
- ad_unit_id - each id denotes a different ad unit (one page can have more than one ad units)
- total_impressions - measurement column measuring the impressions for the particular set of dimensions
- total_revenue - measurement column measuring the revenue for the particular set of dimensions
- viewable_impressions - Number of impressions on the site that were viewable out of all measurable impressions. A display ad is counted as viewable if at least 50% of its area was displayed on screen for at least one second
- measurable_impressions - Impressions that were measurable by Active View out of the total number of eligible impressions. This value should generally be close to 100%. For example, an impression that is rendering in a cross-domain iframe may not be measurable.
- Revenue_share_percent - not every advertiser gives all the revenue to the publisher. They charge a certain share for the services they provide. This captures the fraction of revenue that will actually reach the publishers pocket.

## Import Libraries

In [9]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import spearmanr

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from tensorflow.keras import layers

from google.colab import files

## Data Loading and Exploration

In [6]:
uploaded = files.upload()

Saving dataset-adtech.csv to dataset-adtech.csv


In [7]:
df = pd.read_csv('dataset-adtech.csv')
df.head()

,date,site_id,ad_type_id,geo_id,device_category_id,advertiser_id,order_id,line_item_type_id,os_id,integration_type_id,monetization_channel_id,ad_unit_id,total_impressions,total_revenue,viewable_impressions,measurable_impressions,revenue_share_percent
0,2019-06-30 00:00:00,351,10,187,2,84,3473,19,60,1,4,5174,16,0.0,2,16,1
1,2019-06-30 00:00:00,351,10,187,2,84,3473,19,58,1,4,5174,6,0.0,0,6,1
2,2019-06-30 00:00:00,351,10,147,2,84,3473,19,60,1,4,5174,4,0.0,0,4,1
3,2019-06-30 00:00:00,351,10,147,2,84,3473,19,58,1,4,5174,4,0.0,0,4,1
4,2019-06-30 00:00:00,348,10,147,3,79,3463,20,59,1,19,5138,4,0.0,0,4,1


## Data Transformation and Feature Engineering

## Exploratory Data Analysis (EDA)

## Modeling and Evaluation

## Conclusion and Insights